<a href="https://colab.research.google.com/github/prabakaranc98/mlStreaks/blob/main/CNN_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn  as nn

import torchvision



In [40]:
import torchvision.transforms as transforms

In [98]:
tr = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=(0.5,),std=(0.5,))])

In [99]:
train_dataset = torchvision.datasets.FashionMNIST(root = "./data",transform = tr,train=True,download=True)

In [100]:
import numpy as np
np.array(train_dataset[23][0]).shape

(1, 28, 28)

In [101]:
test_dataset = torchvision.datasets.FashionMNIST(root = "./data",transform = tr,train=False,download=True)

In [102]:
train_dl = torch.utils.data.DataLoader(train_dataset , batch_size=256)
test_dl = torch.utils.data.DataLoader(test_dataset ,
                                      batch_size=32)

In [103]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.layers = nn.Sequential(
        nn.Conv2d(1,10,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(10,10,kernel_size=3,padding=1),
        nn.ReLU() ,
        nn.MaxPool2d(kernel_size=2,stride=2),
        nn.Flatten(),
        nn.Linear(10*14*14,64),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(64,10),
        )

  def forward(self,x):
      x = self.layers(x)
      return x



In [104]:
device = torch.device("cuda")
model = CNN().to(device)

In [105]:
model

CNN(
  (layers): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Flatten(start_dim=1, end_dim=-1)
    (6): Linear(in_features=1960, out_features=64, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [106]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

In [107]:
device
train_loss = []
test_loss = []
accuracy_oa = []


In [108]:
from tqdm import tqdm
num_epochs = 25

for epoch in tqdm(range(num_epochs)):

  batch_test_loss = []
  batch_train_loss = []
  accuracy = []



  model.train()

  for inputs , target in train_dl:
      optimizer.zero_grad()
      inputs , target = inputs.to(device),target.to(device)
      outputs = model(inputs)
      loss = criterion(outputs,target)
      loss.backward()
      optimizer.step()
      batch_train_loss.append(loss.item())


  model.eval()
  with torch.no_grad():
     for inputs , target in test_dl:
        inputs , target = inputs.to(device),target.to(device)

        outputs = model(inputs)
        loss = criterion(outputs,target)
        batch_test_loss.append(loss.item())

  train_loss.append(np.mean(batch_train_loss))
  test_loss.append(np.mean(batch_test_loss))
  accuracy_oa.append(np.mean(accuracy))

  0%|          | 0/25 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

100%|██████████| 25/25 [06:37<00:00, 15.91s/it]


In [109]:
import plotly.express as px
px.line(y=[train_loss,test_loss])